In [150]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_

from box import Box

In [ ]:
"""
num_words: 모델에 입력되는 전체 아이템 수
latent: 모델의 잠재 변수 z의 차원, latent=2048로 설정
hidden: enc, dec dense layer dims
items_sampling: 대규모 아이템 데이터셋을 처리할 때 입력 아이템의 일부만 사용하는 비율, MV20M에서는 ease_items_sampling=0.33로 설정(가장 인기 있는 아이템의 약 33%만 사용)
lr: 1step_lr=0.00005, 2step_lr=0.00001
Focal Loss의 파라미터(fl_alpha, fl_gamma): Focal Loss계산시 사용, fl_alpha=0.25, fl_gamma=2.0로 설정
epochs: 1step = 50, 2step = 20, 3step=20
"""


In [137]:
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)  # Numpy 모듈
    torch.manual_seed(seed_value)  # CPU PyTorch 함수
    
    # CUDA(GPU) PyTorch 시드 고정
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  # 멀티 GPU 사용 시
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False


set_seed(42)

### Preparing Data

In [138]:
# 데이터 로드
data_file = "/opt/ml/input/data/train/train_ratings.csv"
rating_df = pd.read_csv(data_file)

# user와 item에 대한 고유한 인덱스 생성
user_list = rating_df['user'].unique()
item_list = rating_df['item'].unique()

user_to_index = {user: index for index, user in enumerate(user_list)}
item_to_index = {item: index for index, item in enumerate(item_list)}

# 데이터셋에서 user, item을 인덱스로 변환
rating_df['user'] = rating_df['user'].map(user_to_index)
rating_df['item'] = rating_df['item'].map(item_to_index)

num_users = len(user_to_index)
num_items = len(item_to_index)

# csr_matrix 생성
rows = rating_df['user'].values
cols = rating_df['item'].values
# data = np.ones(len(rating_df)) --> 0.1599
rating_score = 1  # parameter
data = np.full(len(rating_df), rating_score)
rating_matrix = csr_matrix((data, (rows, cols)), shape=(num_users, num_items))
dense_rating_matrix = rating_matrix.toarray()

In [139]:
current_time = datetime.now().strftime('%Y%m%d-%H%M%S')
config = {
    'current_time' : current_time,
    
    'data_path' : "/opt/ml/input/data/train" , # 데이터 경로
    'submission_path' : "/data/ephemeral/home/code_ml/output", 
    'model_path' : "/data/ephemeral/home/code_ml/output", # 모델 저장 경로

    'user_to_index' : user_to_index,
    'item_to_index' : item_to_index,

    'num_users' : num_users,
    'num_items' : num_items,
    
    'rating_score' : rating_score,
    
    'rating_matrix' : rating_matrix,   # csr_matrix type
    'dense_rating_matrix' : dense_rating_matrix,   # dense matrix
    
    'batch_size' : 256,
    'latent' : 1024,
    'hidden' : 2048,
    'num_epochs': 25
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = Box(config)

In [140]:
class VASPDataset(Dataset):
    def __init__(self, rating_matrix=config.rating_matrix, num_users=config.num_users):
        self.rating_matrix = rating_matrix
        self.num_users = num_users   # 전체 사용자 수
        self.users = list(range(num_users))

    def __len__(self):
        return self.num_users

    def __getitem__(self, idx):
        user_interaction = self.rating_matrix[idx].toarray()
        user_interaction_tensor = torch.FloatTensor(user_interaction)
        
        return (idx, user_interaction_tensor.squeeze(0))

### Model

In [141]:
############################## For reparameterizing ##############################    
class Sampling(nn.Module):
    """
    reparameterization trick
    """
    def forward(self, inputs):
        z_mean, z_log_var = inputs
        
        # z_mean과 동일한 형태 텐서 생성
        # 분포 내의 실제 z값을 무작위로 선택하는 데 사용(무작위성 부여)
        epsilon = torch.randn_like(z_mean)  # Standard normal distribution ~ N(0,1)
        return z_mean + torch.exp(0.5 * z_log_var) * epsilon

############################## VASP ##############################
class VASPModel(nn.Module):
    def __init__(self, num_words, latent=1024, hidden=1024, items_sampling=1.):
        """
        num_words: 전체 아이템 수 --> same as input dim
        items_sampling: 0~1값을 가짐
        """
        super(VASPModel, self).__init__()
        # self.sampled_items = int(num_words * items_sampling)   # 실제 모델에서 사용될 아이템 수
        
        # # self.sampled_items가 유효한지
        # assert self.sampled_items > 0 and self.sampled_items <= num_words
        # # sampling 활성화 여부
        # self.sampling_enabled = self.sampled_items < num_words

        self.num_items = num_words
        self.latent = latent
        self.hidden = hidden

        # Encoder Layers
        self.encoder_layers = nn.Sequential(
            nn.Linear(self.num_items, self.hidden),
            nn.LayerNorm(self.hidden),
            nn.SiLU(),
            nn.Linear(self.hidden, self.hidden),
            nn.LayerNorm(self.hidden),
            nn.SiLU(),
        )

        # z_mean and z_log_variance for Sampling
        self.dense_mean = nn.Linear(self.hidden, self.latent)
        self.dense_log_var = nn.Linear(self.hidden, self.latent)

        # Sampling Layer
        self.sampling = Sampling()

        # Decoder Layers
        self.decoder_layers = nn.Sequential(
            nn.Linear(self.latent, self.hidden),
            nn.LayerNorm(self.hidden),
            nn.SiLU(),
        )

        # Output Layers for Decoder
        self.decoder_resnet = nn.Linear(self.hidden, self.num_items)
        self.decoder_latent = nn.Linear(self.latent, self.num_items)

        # EASE Layer
        self.ease = nn.Linear(self.num_items, self.num_items, bias=False)
        nn.init.xavier_uniform_(self.ease.weight)
        self.apply_diagonal_zero(self.ease.weight)
        # self.ease.weight.data = self.ease.weight.data * (1 - torch.eye(self.num_items))

    def apply_diagonal_zero(self, weight):
        # 대각행렬 0
        with torch.no_grad():
            weight.fill_diagonal_(0)

    def encode(self, x):
        encoded = self.encoder_layers(x)
        z_mean = self.dense_mean(encoded)
        z_log_var = self.dense_log_var(encoded)
        return z_mean, z_log_var

    def decode(self, z):
        decoded = self.decoder_layers(z)
        """
        self.decoder_resnet(decoded): z로부터 생성된 출력, 비선형 변환
        self.decoder_latent(z): z의 다른 변환
        """
        return self.decoder_resnet(decoded), self.decoder_latent(z)

    def forward(self, user_ratings):
        # if self.sampling_enabled:
        #     sampled_x = x[:, :self.sampled_items]
        #     non_sampled = torch.zeros_like(x[:, self.sampled_items:])
        # else:
        #     sampled_x = x

        # encoding --> z_mean과 z_log_var 생성
        z_mean, z_log_var = self.encode(user_ratings)
        
        # reparameterize
        z = self.sampling((z_mean, z_log_var))
        
        # decode
        decoded_r, decoded_l = self.decode(z)

        ease_out = self.ease(user_ratings)

        # if self.sampling_enabled:
        #     ease_out = torch.cat([ease_out, non_sampled], dim=-1)

        return decoded_r * decoded_l * ease_out


### Loss Function

In [142]:
# MultiVAE
def loss_function_vae(recon_x, x, mu, logvar, anneal=1.0):
    BCE = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
    KLD = -0.5 * torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))
    return BCE + anneal * KLD

# MultiDAE
def loss_function_dae(recon_x, x):
    BCE = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
    return BCE

class SigmoidFocalCrossEntropy(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(SigmoidFocalCrossEntropy, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Sigmoid 함수 적용
        probs = torch.sigmoid(inputs)
        # p_t 계산
        pt = probs * targets + (1 - probs) * (1 - targets)
        # alpha_t 계산
        alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
        # Focal loss 계산
        loss = -alpha_t * (1 - pt).pow(self.gamma) * pt.log()
        
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

### train & inference & submission

In [143]:
def train(model, criterion, optimizer, data_loader, scheduler, clip_value=1.0):
    model.train()
    loss_val = 0
    for idx, (_, user_rating) in enumerate(data_loader):
        user_rating = user_rating.to(device)
        optimizer.zero_grad()
        output  = model(user_rating)
        loss = criterion(output, user_rating)
        loss_val += loss.item()
        loss.backward()
        
        clip_grad_norm_(model.parameters(), clip_value)  ##
        optimizer.step()
        print(f"loss: {loss.item()}")
    scheduler.step()  ##
    loss_val /= len(data_loader)
    print(f"loss_val: {loss_val}")
    return loss_val

def predict(model, data_loader, 
            dense_rating_matrix = config.dense_rating_matrix, 
            N=10):
    model.eval()
    pred_list = None
    answer_list = None

    with torch.no_grad(): 
        for i, (user_ids, user_ratings) in enumerate(data_loader):
            user_ratings = user_ratings.to(device)
            rating_pred = model(user_ratings)
            batch_user_index = user_ids.cpu().numpy()
            
            for idx, user_idx in enumerate(batch_user_index):
                rated_indices = np.where(dense_rating_matrix[user_idx] > 0)[0]
                rating_pred[idx, rated_indices] = -np.inf
            
            # 각 사용자별로 상위 10개 아이템의 인덱스 추출
            ind = np.argpartition(rating_pred.cpu().numpy(), -N)[:, -N:]
            arr_ind = rating_pred[np.arange(len(rating_pred))[:, None], ind].cpu().numpy()
            arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::-1]
            batch_pred_list = ind[np.arange(len(rating_pred))[:, None], arr_ind_argsort]

            if i == 0:
                pred_list = batch_pred_list
            else:
                pred_list = np.append(pred_list, batch_pred_list, axis=0)
    
    return pred_list

def submission(pred_list, 
               item_to_index=config.item_to_index, 
               submission_path=config.submission_path, 
               current_time=config.current_time):
    index_to_item = {index: item for item, index in item_to_index.items()}
    top_n_items_per_user_ids = [[index_to_item[idx] for idx in user_items] for user_items in pred_list]
    
    result = []
    for user_id, items in zip(user_list, top_n_items_per_user_ids):
        for item_id in items:
            result.append((user_id, item_id))
    args_str = f"submission_VASP_rs{config.rating_score}_tm{current_time}.csv"
    checkpoint_path = os.path.join(submission_path, args_str)
    submission_df = pd.DataFrame(result, columns=['user', 'item'])
    submission_df.to_csv(checkpoint_path, index=False)

### Main

In [144]:
vasp_dataset = VASPDataset(
    num_users = config.num_users,
    )

data_loader = DataLoader(
    vasp_dataset,
    batch_size = config.batch_size, 
    shuffle = True, 
    pin_memory = True,
    )

model = VASPModel(num_words=config.num_items).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.00005)
criterion = SigmoidFocalCrossEntropy()

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
for epoch in range(1, config.num_epochs + 1):
    tbar = tqdm(range(1))
    for _ in tbar:
        train_loss = train(
            model = model, 
            criterion = criterion, 
            optimizer = optimizer, 
            data_loader = data_loader,
            scheduler = scheduler)

In [146]:
args_str = f"VASP_rs{config.rating_score}_tm{current_time}.pt"
checkpoint_path = os.path.join(config.model_path, args_str)

# 저장
torch.save(model.state_dict(), checkpoint_path)

In [120]:
model = VASPModel(num_words=config.num_items)
saved_path = checkpoint_path
model.load_state_dict(torch.load(saved_path))

<All keys matched successfully>

In [ ]:
predict_data_loader = DataLoader(
    vasp_dataset,
    batch_size = config.batch_size, 
    shuffle = False, 
    pin_memory = True,
    )

In [147]:
submit_df = predict(model, predict_data_loader, config.dense_rating_matrix, 10)

In [149]:
submission(submit_df, config.item_to_index,
           config.submission_path,
           config.current_time)